In [2]:
# The following is to know when this notebook has been run and with which python version.
import time, sys
print(time.ctime())
print(sys.version.split('|')[0])

Mon Aug 21 15:50:52 2023
3.9.16 (main, Jan 11 2023, 10:02:19) 
[Clang 14.0.6 ]


# C: How to read and write files (ASCII and FITS)

This is part of the Python lecture given by Christophe Morisset at IA-UNAM.

Some informations are here: http://www.tutorialspoint.com/python/python_files_io.htm

## Reading a simple ascii file

In [3]:
# numpy is needed in some part of the lecture
import numpy as np

First of all, we will have to have some files on the hard drive to read them The following notebook cell will write a file in the same directory where the notebook has been started.

In [4]:
%%writefile data1.dat
1   2.3  6   8 star
2   3.5  7   9 galaxy
3  -4.2  5   7 cluster

Overwriting data1.dat


Now the goal is to read this file. The first way is to open the file, read it completely in a variable and close the file. Then we can play with the content of the file.

In [5]:
datafile = open('data1.dat', 'r') # Open the file to read it

In [6]:
data = datafile.readlines() # The variable data will receive the content of the file.

In [7]:
datafile.close() # Not need anymore of the file.

In [8]:
print(type(data)) # The data file is stored in the form of a list, each element of the list corresponding to a row of the list.

<class 'list'>


In [9]:
print(data) # Each row is a string and terminates with \n, symbol of END OF LINE.

['1   2.3  6   8 star\n', '2   3.5  7   9 galaxy\n', '3  -4.2  5   7 cluster\n']


In [10]:
print(len(data)) # number of rows

3


In [12]:
print(data[0], 'tralala', 'trololo')

1   2.3  6   8 star
 tralala trololo


In [13]:
for row in data:
    print(row)

1   2.3  6   8 star

2   3.5  7   9 galaxy

3  -4.2  5   7 cluster



In [15]:
# In python 3:
for row in data:
    print(row, end='')

1   2.3  6   8 star
2   3.5  7   9 galaxy
3  -4.2  5   7 cluster


In [16]:
print(type(data[0])) # Each element is a string

<class 'str'>


Now it is easy to separate each field with the split command: 

In [17]:
for row in data:
    print(row.split())

['1', '2.3', '6', '8', 'star']
['2', '3.5', '7', '9', 'galaxy']
['3', '-4.2', '5', '7', 'cluster']


In [18]:
# One can also transform the data if the type is known:
for row in data:
    this_data = row.split()
    print('N = {0:2d} f = {1:5.2f} type = {2:>10s}'.format(int(this_data[0]), 
                                                           float(this_data[1]), 
                                                           this_data[4]))

N =  1 f =  2.30 type =       star
N =  2 f =  3.50 type =     galaxy
N =  3 f = -4.20 type =    cluster


In [19]:
# One can even fill a list with the data, by column:
N = []
f = []
type_ = [] # take care, type is a python command, you can erase it if you use it...
for row in data:
    this_data = row.split()
    N.append(int(this_data[0]))
    f.append(float(this_data[1]))
    type_.append(this_data[4])
print(N)
print(f)
print(type_)
N = np.array(N)
print(N)

[1, 2, 3]
[2.3, 3.5, -4.2]
['star', 'galaxy', 'cluster']
[1 2 3]


In [20]:
# If the file number of rows is not too big, you can use list comprehension (and even send the result to a numpy array)
N = np.array([int(row.split()[0]) for row in data])
f = np.array([float(row.split()[1]) for row in data])
# Each one of this command scans all the rows, don't use for huge files
print(N)
print(f)

[1 2 3]
[ 2.3  3.5 -4.2]


## How to treat special rows (headers, comments)

In [28]:
%%writefile data2.dat
# The following data are for test purpose
N    f   x   y type
1   2.3  6   8 star
2   3.5  7   9 galaxy
3  -4.2  5   7 cluster
#4  -10.5  5  7 test

Overwriting data2.dat


In [29]:
!cat data2.dat # Just to check that the # comments are also in the file

# The following data are for test purpose
N    f   x   y type
1   2.3  6   8 star
2   3.5  7   9 galaxy
3  -4.2  5   7 cluster
#4  -10.5  5  7 test


The file has to be read row by row, to be sure that special cases are treated.

In [25]:
datafile = open('data2.dat', 'r') # Open the file to read it

row = datafile.readline() # this reads only one line
first_comment = row
print(first_comment, end='')

row = datafile.readline() # this reads only one line
header = row
print(header, end='')

data = []
while True: # loops until exit by break command
    row = datafile.readline()
    if row == '':
        break
    if row[0] != '#' and row[0] != '\n': # comment lines are skipped
        data.append(row)
datafile.close()
print(data)

# The following data are for test purpose
N    f   x   y type
['1   2.3  6   8 star\n', '2   3.5  7   9 galaxy\n', '3  -4.2  5   7 cluster\n']


In [30]:
datafile = open('data2.dat', 'r') # Open the file to read it
row = datafile.readline() # this reads only one line
first_comment = row
print(first_comment, end='')
row = datafile.readline() # this reads only one line
header = row
print(header, end='')
data = []
row = datafile.readline()
while row != '': # ends at end of the file
    if row[0] != '#': # comment lines are skipped
        data.append(row)
    row = datafile.readline()
datafile.close()
print(data)

# The following data are for test purpose
N    f   x   y type
['1   2.3  6   8 star\n', '2   3.5  7   9 galaxy\n', '3  -4.2  5   7 cluster\n']


In [31]:
# very shorter way to deal with the file. No need to look for the end of the file.
datafile = open('data2.dat', 'r') # Open the file to read it
data = []
for row in datafile:
    if row[0] != '#': # comment lines are skipped
        data.append(row)  
datafile.close()
print(data)
# This way will include the header in the data... Not what we want

['N    f   x   y type\n', '1   2.3  6   8 star\n', '2   3.5  7   9 galaxy\n', '3  -4.2  5   7 cluster\n']


In [32]:
# very shorter way to deal with the file:
# we know that the header is the first no-comment line in the file.
datafile = open('data2.dat', 'r') # Open the file to read it
data = []
comments = [] # we can keep the comments for some usage
header_not_read = True # We will turn it to True once the header is read
for row in datafile:
    if row[0] != '#': # comment lines are skipped
        if header_not_read:
            header = row
            header_not_read = False # next time, data will be read
        else:
            data.append(row)
    else:
        comments.append(row)
datafile.close()
print(header, end='')
print('-----')
print(data)
print('-----')
print(comments)

N    f   x   y type
-----
['1   2.3  6   8 star\n', '2   3.5  7   9 galaxy\n', '3  -4.2  5   7 cluster\n']
-----
['# The following data are for test purpose\n', '#4  -10.5  5  7 test\n']


In [33]:
# Alternative way using "with". No need to close the file, done when the "with" block is terminated.
data = []
comments = []
header_read = False
def change_type(row_split):
    # This function change the type of the data read from the file from 5 strings into int, 3 floats and a string
    # It also return the result in form of a tuple
    return (int(row_split[0]), 
            float(row_split[1]), 
            float(row_split[2]), 
            float(row_split[3]), 
            row_split[4])
with open('data2.dat', 'r') as datafile:
    for row in datafile:
        if row[0] != '#' and row[0] != '\n': # comment lines are skipped
            if not header_read:
                header = row
                header_read = True
            else:
                data.append(change_type(row.split()))
        else:
            comments.append(row)
print(header)
print(data)
print(comments)

N    f   x   y type

[(1, 2.3, 6.0, 8.0, 'star'), (2, 3.5, 7.0, 9.0, 'galaxy'), (3, -4.2, 5.0, 7.0, 'cluster')]
['# The following data are for test purpose\n', '#4  -10.5  5  7 test\n']


In [34]:
# We can define the result as a structured array
# We use the header to define the field names.
# data must be a list of tuples.
a = np.array(data, dtype={'names':header.split(), 
                          'formats':['i4','f16', 'f16', 'f16', 'U10']})

In [35]:
a

array([(1,  2.3, 6., 8., 'star'), (2,  3.5, 7., 9., 'galaxy'),
       (3, -4.2, 5., 7., 'cluster')],
      dtype=[('N', '<i4'), ('f', '<f16'), ('x', '<f16'), ('y', '<f16'), ('type', '<U10')])

In [36]:
print(data[0])

(1, 2.3, 6.0, 8.0, 'star')


In [37]:
print(a[0])

(1, 2.3, 6., 8., 'star')


In [38]:
# Easy access to the columns, by their name
print(a['N'])

[1 2 3]


In [39]:
print(a['type'])

['star' 'galaxy' 'cluster']


In [40]:
# Easy combine the values of columns
print(np.sqrt(a['x']**2 + a['y']**2))

[10.         11.40175425  8.60232527]


### Using numpy loadtxt

http://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html

In [41]:
# Fast way for reading the file
# One hace to tell to skip the 2 first rows
# skiprows 
b = np.loadtxt('data2.dat', skiprows=2, dtype='i4,f, f, f, U10')

In [42]:
print(b)

[(1,  2.3, 6., 8., 'star') (2,  3.5, 7., 9., 'galaxy')
 (3, -4.2, 5., 7., 'cluster')]


In [43]:
type(b)

numpy.ndarray

In [44]:
# The names of the columns are f0, f1, f2, etc
b.dtype

dtype([('f0', '<i4'), ('f1', '<f4'), ('f2', '<f4'), ('f3', '<f4'), ('f4', '<U10')])

### Using numpy genfromtxt

http://docs.scipy.org/doc/numpy/reference/generated/numpy.genfromtxt.html

In [45]:
# Fast and versatile way to read the file
# the names are taken from the file
# The types are defined automatically when reading the columns
c = np.genfromtxt('data2.dat', names=True, dtype=None, skip_header=1)

/var/folders/7b/7gktm_g91hn54p3gxj15kb1m0000gn/T/ipykernel_28514/3267449474.py:4: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  c = np.genfromtxt('data2.dat', names=True, dtype=None, skip_header=1)


In [46]:
print(c)

[(1,  2.3, 6, 8, b'star') (2,  3.5, 7, 9, b'galaxy')
 (3, -4.2, 5, 7, b'cluster')]


In [47]:
type(c)

numpy.ndarray

In [48]:
c.dtype

dtype([('N', '<i8'), ('f', '<f8'), ('x', '<i8'), ('y', '<i8'), ('type', 'S7')])

In [49]:
c['f']

array([ 2.3,  3.5, -4.2])

Now a value of x is missing (not possible with space separator, so we use "," as separator):

In [50]:
%%writefile data3.dat
# The following data are for test purpose
N,    f,   x,   y, type
1,   2.3,  6,   8, star
2,   3000.5,   ,  9, galaxy
#3,  -4.2,  5,   7, cluster
4,  -10.5,  5,  7, test

Overwriting data3.dat


In [51]:
d = np.genfromtxt('data3.dat', names=True, dtype=None, skip_header=1, 
                  delimiter=',')

/var/folders/7b/7gktm_g91hn54p3gxj15kb1m0000gn/T/ipykernel_28514/1151990606.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  d = np.genfromtxt('data3.dat', names=True, dtype=None, skip_header=1,


In [52]:
# The missing value has been changed to -1
d

array([(1,  2.3000e+00,  6, 8, b' star'),
       (2,  3.0005e+03, -1, 9, b' galaxy'),
       (4, -1.0500e+01,  5, 7, b' test')],
      dtype=[('N', '<i8'), ('f', '<f8'), ('x', '<i8'), ('y', '<i8'), ('type', 'S7')])

In [58]:
# Th emissing value can be set to whatever you want (but non a NaN here, as the typ eis integer, and NaN is a float...)
d = np.genfromtxt('data3.dat', names=True, dtype=None, skip_header=1, delimiter=',', 
                  filling_values=999)

/var/folders/7b/7gktm_g91hn54p3gxj15kb1m0000gn/T/ipykernel_28514/612411852.py:2: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  d = np.genfromtxt('data3.dat', names=True, dtype=None, skip_header=1, delimiter=',',


In [59]:
d['x'][1]

999

In [60]:
# ons can select the columns to be store
e = np.genfromtxt('data3.dat', names=True, dtype=None, skip_header=1, 
                  delimiter=',',usecols=(0,1,4))

/var/folders/7b/7gktm_g91hn54p3gxj15kb1m0000gn/T/ipykernel_28514/699879124.py:2: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  e = np.genfromtxt('data3.dat', names=True, dtype=None, skip_header=1,


In [63]:
print(e)

[(1,  2.3000e+00, b' star') (2,  3.0005e+03, b' galaxy')
 (4, -1.0500e+01, b' test')]


In [64]:
# one can select the columns to be store
N, f, typ = np.genfromtxt('data3.dat', skip_header=2,
                  delimiter=',',usecols=(0,1,4), unpack=True)

In [65]:
# The resulting array now contains only the given columns
print(N)
print(f)

[1. 2. 4.]
[ 2.3000e+00  3.0005e+03 -1.0500e+01]


### Using recfrom to obtain a record array

In [66]:
# Uses the same keywords than genfromtxt
f = np.recfromtxt('data3.dat', names=True, dtype=None, skip_header=1, 
                  delimiter=',',usecols=("N", "f", "type"))

/Users/christophemorisset/anaconda3/envs/ML/lib/python3.9/site-packages/numpy/lib/npyio.py:2484: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


In [67]:
f

rec.array([(1,  2.3000e+00, b' star'), (2,  3.0005e+03, b' galaxy'),
           (4, -1.0500e+01, b' test')],
          dtype=[('N', '<i8'), ('f', '<f8'), ('type', 'S7')])

In [68]:
f.N

array([1, 2, 4])

In [69]:
f['N']

array([1, 2, 4])

## Fixed size ascii files

In [70]:
%%writefile data4.dat
#  Line      Iobs    lambda  relat_error Obs_code
H  1  4861A 1.00000    4861. 0.08000  Anabel                               
H  1  6563A 2.8667     6563. 0.19467  Anabel                               
H  1  4340A 0.4933     4340. 0.03307  Anabel                               
H  1  4102A 0.2907     4102. 0.02229  Anabel                               
H  1  3970A 0.1800     3970. 0.01253  Anabel                               
N  2  6584A 2.1681     6584. 0.08686  Anabel                               
N  2 121.7m 0.0044621217000. 0.20000  Liu                                  
O  1  6300A 0.0147     6300. 0.00325  Anabel                               
TOTL  2326A 0.07900    2326. 0.20000  Adams                                
C  2 157.6m 0.00856 1576000. 0.20000  Liu                                  
O  1 63.17m 0.13647  631700. 0.10000  Liu                                  
O  1 145.5m 0.00446 1455000. 0.200    Liu                                  
TOTL  3727A 0.77609    3727. 0.200    Torres-Peimbert                      
S II  4070A 0.06174    4070. 0.200    Torres-Peimbert                      
S II  4078A 0.06174    4078. 0.200    Torres-Peimbert                      

Overwriting data4.dat


In [71]:
# Here we cannot use SPACE as a separator, as some strings contains spaces.
# "delimiter" is used to specify the size (in characters in the file) of each variables. 
# The types must be clearly defined too.
obs  = np.genfromtxt('data4.dat', 
                     dtype=["U11","float","float","float","U2"],
                     delimiter=[11,9,8,10,2],
                     names = True
                     )

In [72]:
obs # The same delimiter (fixed sizes) is applied to the names. May not be what you want:

array([('H  1  4861A', 1.      ,    4861., 0.08   , 'An'),
       ('H  1  6563A', 2.8667  ,    6563., 0.19467, 'An'),
       ('H  1  4340A', 0.4933  ,    4340., 0.03307, 'An'),
       ('H  1  4102A', 0.2907  ,    4102., 0.02229, 'An'),
       ('H  1  3970A', 0.18    ,    3970., 0.01253, 'An'),
       ('N  2  6584A', 2.1681  ,    6584., 0.08686, 'An'),
       ('N  2 121.7m', 0.004462, 1217000., 0.2    , 'Li'),
       ('O  1  6300A', 0.0147  ,    6300., 0.00325, 'An'),
       ('TOTL  2326A', 0.079   ,    2326., 0.2    , 'Ad'),
       ('C  2 157.6m', 0.00856 , 1576000., 0.2    , 'Li'),
       ('O  1 63.17m', 0.13647 ,  631700., 0.1    , 'Li'),
       ('O  1 145.5m', 0.00446 , 1455000., 0.2    , 'Li'),
       ('TOTL  3727A', 0.77609 ,    3727., 0.2    , 'To'),
       ('S II  4070A', 0.06174 ,    4070., 0.2    , 'To'),
       ('S II  4078A', 0.06174 ,    4078., 0.2    , 'To')],
      dtype=[('Line', '<U11'), ('Iobs', '<f8'), ('lambda', '<f8'), ('relat_erro', '<f8'), ('r', '<U2')])

In [73]:
# Defining the names:
obs2  = np.genfromtxt('data4.dat', skip_header=1,
                     dtype=None,
                     delimiter=[11,9,8,10,2],
                     names = ['label', 'i_obs', 'lambda', 'e_obs', 'observer']
                     )

/var/folders/7b/7gktm_g91hn54p3gxj15kb1m0000gn/T/ipykernel_28514/389876475.py:2: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  obs2  = np.genfromtxt('data4.dat', skip_header=1,


In [74]:
obs2 

array([(b'H  1  4861A', 1.      ,    4861., 0.08   , b'An'),
       (b'H  1  6563A', 2.8667  ,    6563., 0.19467, b'An'),
       (b'H  1  4340A', 0.4933  ,    4340., 0.03307, b'An'),
       (b'H  1  4102A', 0.2907  ,    4102., 0.02229, b'An'),
       (b'H  1  3970A', 0.18    ,    3970., 0.01253, b'An'),
       (b'N  2  6584A', 2.1681  ,    6584., 0.08686, b'An'),
       (b'N  2 121.7m', 0.004462, 1217000., 0.2    , b'Li'),
       (b'O  1  6300A', 0.0147  ,    6300., 0.00325, b'An'),
       (b'TOTL  2326A', 0.079   ,    2326., 0.2    , b'Ad'),
       (b'C  2 157.6m', 0.00856 , 1576000., 0.2    , b'Li'),
       (b'O  1 63.17m', 0.13647 ,  631700., 0.1    , b'Li'),
       (b'O  1 145.5m', 0.00446 , 1455000., 0.2    , b'Li'),
       (b'TOTL  3727A', 0.77609 ,    3727., 0.2    , b'To'),
       (b'S II  4070A', 0.06174 ,    4070., 0.2    , b'To'),
       (b'S II  4078A', 0.06174 ,    4078., 0.2    , b'To')],
      dtype=[('label', 'S11'), ('i_obs', '<f8'), ('lambda', '<f8'), ('e_obs', '<f8')

In [76]:
%%writefile data5.dat
#  Line      Iobs    lambda  relat_error Obs_code
H  1  4861A 1.00000    4861. 0.08000 x Anabel                               
H  1  6563A 2.8667     6563. 0.19467 x Anabel                               
H  1  4340A 0.4933     4340. 0.03307 x Anabel                               
H  1  4102A 0.2907     4102. 0.02229 x Anabel                               
H  1  3970A 0.1800     3970. 0.01253 t Anabel                               
N  2  6584A 2.1681           0.08686 x Anabel                               
N  2 121.7m 0.00446 1217000. 0.20000 g Liu                                  
O  1  6300A 0.0147     6300. 0.00325 t Anabel                               
TOTL  2326A 0.07900    2326. 0.20000 g Adams                                
C  2 157.6m 0.00856 1576000. 0.20000 t Liu                                  
O  1 63.17m 0.13647  631700. 0.10000 g Liu                                  
O  1 145.5m 0.00446 1455000. 0.200   g Liu                                  
TOTL  3727A 0.77609    3727. 0.200   g Torres-Peimbert                      
S II  4070A 0.06174    4070. 0.200   g Torres-Peimbert                      
S II  4078A 0.06174    4078. 0.200   g Torres-Peimbert   

Overwriting data5.dat


In [77]:
# Here we want to skip one column:
obs3  = np.genfromtxt('data5.dat', skip_header=1,
                     dtype=None,
                     delimiter=[11, 8, 9, 9, 2, 2],
                     names = ['label', 'i_obs', 'lambda', 'e_obs', 'na', 'observer'],
                     usecols = (0, 1, 2, 3, 5)
                     )

/var/folders/7b/7gktm_g91hn54p3gxj15kb1m0000gn/T/ipykernel_28514/3725919490.py:2: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  obs3  = np.genfromtxt('data5.dat', skip_header=1,


In [78]:
obs3

array([(b'H  1  4861A', 1.     ,    4861., 0.08   , b'An'),
       (b'H  1  6563A', 2.8667 ,    6563., 0.19467, b'An'),
       (b'H  1  4340A', 0.4933 ,    4340., 0.03307, b'An'),
       (b'H  1  4102A', 0.2907 ,    4102., 0.02229, b'An'),
       (b'H  1  3970A', 0.18   ,    3970., 0.01253, b'An'),
       (b'N  2  6584A', 2.1681 ,      nan, 0.08686, b'An'),
       (b'N  2 121.7m', 0.00446, 1217000., 0.2    , b'Li'),
       (b'O  1  6300A', 0.0147 ,    6300., 0.00325, b'An'),
       (b'TOTL  2326A', 0.079  ,    2326., 0.2    , b'Ad'),
       (b'C  2 157.6m', 0.00856, 1576000., 0.2    , b'Li'),
       (b'O  1 63.17m', 0.13647,  631700., 0.1    , b'Li'),
       (b'O  1 145.5m', 0.00446, 1455000., 0.2    , b'Li'),
       (b'TOTL  3727A', 0.77609,    3727., 0.2    , b'To'),
       (b'S II  4070A', 0.06174,    4070., 0.2    , b'To'),
       (b'S II  4078A', 0.06174,    4078., 0.2    , b'To')],
      dtype=[('label', 'S11'), ('i_obs', '<f8'), ('lambda', '<f8'), ('e_obs', '<f8'), ('observer', 

In [79]:
obs3['lambda']

array([   4861.,    6563.,    4340.,    4102.,    3970.,      nan,
       1217000.,    6300.,    2326., 1576000.,  631700., 1455000.,
          3727.,    4070.,    4078.])

In [80]:
new_obs3 = obs3.view(np.recarray)

In [81]:
new_obs3.label

array([b'H  1  4861A', b'H  1  6563A', b'H  1  4340A', b'H  1  4102A',
       b'H  1  3970A', b'N  2  6584A', b'N  2 121.7m', b'O  1  6300A',
       b'TOTL  2326A', b'C  2 157.6m', b'O  1 63.17m', b'O  1 145.5m',
       b'TOTL  3727A', b'S II  4070A', b'S II  4078A'], dtype='|S11')

In [82]:
new_obs3.lambda # lambda is reserved!!!

SyntaxError: invalid syntax (1466917404.py, line 1)

In [83]:
new_obs3['lambda']

array([   4861.,    6563.,    4340.,    4102.,    3970.,      nan,
       1217000.,    6300.,    2326., 1576000.,  631700., 1455000.,
          3727.,    4070.,    4078.])

Using masks on the structured array.

In [84]:
mask_observer = (obs3['observer'] == b'An') & (np.isfinite(obs3['lambda']))
print(obs3[mask_observer])

[(b'H  1  4861A', 1.    , 4861., 0.08   , b'An')
 (b'H  1  6563A', 2.8667, 6563., 0.19467, b'An')
 (b'H  1  4340A', 0.4933, 4340., 0.03307, b'An')
 (b'H  1  4102A', 0.2907, 4102., 0.02229, b'An')
 (b'H  1  3970A', 0.18  , 3970., 0.01253, b'An')
 (b'O  1  6300A', 0.0147, 6300., 0.00325, b'An')]


In [85]:
for o in obs3[mask_observer]:
    print('line {0[label]:4s}, wavelength={0[lambda]}A Intensity={0[i_obs]:5.3f} +/- {1:4.1f}%)'.format(o, o['e_obs']*100))

line b'H  1  4861A', wavelength=4861.0A Intensity=1.000 +/-  8.0%)
line b'H  1  6563A', wavelength=6563.0A Intensity=2.867 +/- 19.5%)
line b'H  1  4340A', wavelength=4340.0A Intensity=0.493 +/-  3.3%)
line b'H  1  4102A', wavelength=4102.0A Intensity=0.291 +/-  2.2%)
line b'H  1  3970A', wavelength=3970.0A Intensity=0.180 +/-  1.3%)
line b'O  1  6300A', wavelength=6300.0A Intensity=0.015 +/-  0.3%)


## Writing files

### Simple "write" method from "open" class

In [86]:
f = open('data10.dat', 'w')

In [87]:
f.write('tralala')
f.write('trololo')

7

In [88]:
!cat 'data10.dat' # the writing method put everything together.

In [89]:
f.close()

In [90]:
!cat 'data10.dat' # the writing method put everything together.

tralalatrololo

In [92]:
f = open('data11.dat', 'w')
f.write('tralala\n') # \n to indicate end of line
f.write('trololo\n')
f.close()
!cat 'data11.dat'

tralala
trololo


In [93]:
f = open('data11.dat', 'a') # Append to the edn of the file
f.write('trilili\n') # \n to indicate end of line
f.write('trululu\n')
f.close()
!cat 'data11.dat'

tralala
trololo
trilili
trululu


In [95]:
a = 'Smith'
b = 3
with open('data12.dat', 'w') as datafile:
    datafile.write("""Hola Sr. {0}
         This is a file # this does not close the "with" command
with a lot of lines.
It is easy to write it.
The value of your data is {1}.
""".format(a, b))

print('end') # This one ends the indentation
    
!cat "data12.dat"

end
Hola Sr. Smith
         This is a file # this does not close the "with" command
with a lot of lines.
It is easy to write it.
The value of your data is 3.


### Using pickle (and cpickle) python specific format

In [ ]:
# Let's define some stuffs we want to keep in a file (data and variable names)
a = 5
b = 'Hola'
c = np.array([1,2,3,4,5])
def d(x):
    """ Function mia"""
    return x**2

In [ ]:
import pickle # The module we will use for this

In [ ]:
pickle.dump((a,b,c,d), open('Demo.pickle','wb')) # Writing the variables

In [ ]:
res = pickle.load(open('Demo.pickle', 'rb'))

In [ ]:
type(res)

In [ ]:
print(res[0])
print(res[1])
print(res[2])

In [ ]:
res[3](5)

In [ ]:
a2,b2,c2,d2 = pickle.load(open('Demo.pickle', 'rb'))

In [ ]:
a2

In [ ]:
d2(10)

In [ ]:
help(d2)

In [ ]:
%timeit res = pickle.load(open('Demo.pickle', 'rb'))

In [ ]:
import gzip
pickle.dump((a,b,c,d), gzip.open('Demo.pklz','wb')) # Writing the variables

In [ ]:
with gzip.open('Demo.pklz','rb') as f:
    a, b, c, d = pickle.load(f)

### FITS files

In [ ]:
import astropy
print(astropy.__version__)

https://docs.astropy.org/en/stable/io/fits/index.html

In [ ]:
from astropy.io import fits

We will use one FITS files from San Pedro Martir echelle spectrograph. The file can be downloaded from: https://github.com/Morisset/Python-lectures-Notebooks/raw/master/Notebooks/n10017o.fits

In [ ]:
hdulist = fits.open('n10017o.fits')

In [ ]:
# The result hdulist is a list of HDU objects. 
# In the case of a simple file, there is only one primary HDU so the list contains only one element
len(hdulist)

In [ ]:
# The information on what the file contains can be obtained by calling the info() method:
hdulist.info()
# The table said that there is only a primary HDU which contains 2154 X 2048 image with data stored in 2 bytes (16 bits) integers.

In [ ]:
# As described above, the HDU (header/data unit) contains header and data. The header is a dictionary. 
# To see what keywords were used in the header one can do:
list(hdulist[0].header.keys())

In [ ]:
# and to get the value of a given keyword :
hdulist[0].header['OBJECT']

In [ ]:
hh = hdulist[0].header
hh?

In [ ]:
hh

In [ ]:
# The header can be printed as it appears in the file by
print(hdulist[0].header.cards)

In [ ]:
# The data in the file are accessible with
data = hdulist[0].data

In [ ]:
data.shape

In [ ]:
type(data)

In [ ]:
# and can be seen with [we need to import matplotlib.pyplot as plt before running this]:
%matplotlib inline
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(12,12))
ax.imshow(data)

In [ ]:
# A column from the data can be plotted with 
plt.plot(data[:,1000])
# where I am plotting the column number 1000. 

In [ ]:
# In the same way a line from the data is plotted with: 
plt.plot(data[1000,:])

In [ ]:
# For this example I'll use a spectrum obtain with the high dispersion camera on board of IUE. 
# The file is opened as usual:
hdulist = fits.open('swp04345.mxhi')

The file is there: https://github.com/Morisset/Python-lectures-Notebooks/raw/master/Notebooks/swp04345.mxhi

In [ ]:
#but now hdulist has 2 elements (2 header/data units):
len(hdulist)

In [ ]:
# We can see that the primary header has dimension (), son does not contain any data. 
# The data are in the extension.
hdulist.info()

In [ ]:
# The first header contains the minimal infirmation:
print(hdulist[0].header.cards)

In [ ]:
# The number of axis is 0 which means there is no data block in the primary HDU. 
# The header of the second HDU begins with the keyword XTENSION and with the specification of the data
print(hdulist[1].header.cards[:5])

In [ ]:
# To progress further we need to know what is in the table. 
# As usual, the columns have names and type of the stored data. 
# These information can be obtained using the column attribute of hdulist:
cols = hdulist[1].columns

In [ ]:
# the cols.info returns the names of the columns and the information of their format and units.
cols.info

In [ ]:
# The data are available using (this example is NOT the right way of plotting the data, it's just an example) 
# and don't forget to import numpy as np to have np.arange working]:

data1 = hdulist[1].data
DTs =  data1.ABS_CAL
WLs = data1.WAVELENGTH
DWs = data1.DELTAW
for WL, DW, DT in zip(WLs, DWs, DTs):
    plt.plot(WL + np.arange(len(DT)) * DW, DT)

### Writing FITS files

In [ ]:
# Creation of numpy array with the data. 
x = np.arange(100)

In [ ]:
# Creation of the HDU from the data. 
hdu = fits.PrimaryHDU(x)
print(hdu.header.cards)

In [ ]:
#Adding additional keywords to the header. 
# The automatically created header contains only the required minimum of keywords. 
# If additional keywords are needed they are added with:
hdu.header['testkey'] = (0.001,'some test value')

In [ ]:
print(hdu.header.cards)

In [ ]:
hdulist = fits.HDUList([hdu])
hdulist.writeto('new.fits', overwrite=True) 
hdulist.close()

Another way to deal with FITS tables is to use the ATpy library, we'll see this later